In [ ]:
#Kết nối GG-Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/Shareddrives/Dehusked_Rice
!python -m pip install pyyaml
import sys, os, distutils.core
#!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

/content/drive/Shareddrives/Dehusked_Rice
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 973.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=a2a1e80d7df0d342a32888171c8f348699b0fbd1a42d630efed162f07286cd3b
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=b8948a609344c1262ed128ed6e4b7efbd9bde587117254f23113973672542127
  Stored in direc

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
torch:  2.2 ; cuda:  cu121
detectron2: 0.6


In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from matplotlib import pyplot as plt
from detectron2.engine import DefaultTrainer
import yaml
from detectron2.utils.visualizer import ColorMode
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

## Split 1

In [ ]:
register_coco_instances("my_dataset_train",{},"/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split1/kfold_split_1.coco/train/_annotations.coco.json","/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split1/kfold_split_1.coco/train")
register_coco_instances("my_dataset_val",{},"/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split1/kfold_split_1.coco/valid/_annotations.coco.json","/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split1/kfold_split_1.coco/valid")
train_metadata = MetadataCatalog.get("my_dataset_train")
train_dataset_dicts = DatasetCatalog.get("my_dataset_train")
val_metadata = MetadataCatalog.get("my_dataset_val")
val_dataset_dicts = DatasetCatalog.get("my_dataset_val")

[05/05 01:06:34 d2.data.datasets.coco]: Loading /content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split1/kfold_split_1.coco/train/_annotations.coco.json takes 1.09 seconds.
WARNING [05/05 01:06:34 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[05/05 01:06:34 d2.data.datasets.coco]: Loaded 216 images in COCO format from /content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split1/kfold_split_1.coco/train/_annotations.coco.json
WARNING [05/05 01:06:34 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[05/05 01:06:34 d2.data.datasets.coco]: Loaded 54 images in COCO format from /content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split1/kfold_split_1.coco/valid/_annotations.coco.json


In [ ]:
Save_detectron2='/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split1/kfold_split_1.coco/result-detectron2'
if not os.path.exists(Save_detectron2):
    os.makedirs(Save_detectron2)
%cd /content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split1/kfold_split_1.coco/result-detectron2
output_dir='/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split1/kfold_split_1.coco/result-detectron2/OUTPUT'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
cfg = get_cfg()
cfg.OUTPUT_DIR = output_dir
cfg.merge_from_file("/content/drive/Shareddrives/Dehusked_Rice/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 5000   # 1000 iterations seems good enough for this dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # Default is 512, using 256 for this dataset.
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) #Create an instance of of DefaultTrainer with the given congiguration
trainer.resume_or_load(resume=False)

/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split1/kfold_split_1.coco/result-detectron2
[05/05 01:13:25 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(

model_final_f10217.pkl: 178MB [00:00, 193MB/s]                           
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


In [ ]:
trainer.train()

[05/05 01:14:06 d2.engine.train_loop]: Starting training from iteration 0


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[05/05 01:14:25 d2.utils.events]:  eta: 1:06:56  iter: 19  total_loss: 3.882  loss_cls: 1.542  loss_box_reg: 0.5506  loss_mask: 0.6905  loss_rpn_cls: 1.008  loss_rpn_loc: 0.08111    time: 0.7822  last_time: 0.7751  data_time: 0.2262  last_data_time: 0.1447   lr: 4.9953e-06  max_mem: 3023M
[05/05 01:14:45 d2.utils.events]:  eta: 1:06:04  iter: 39  total_loss: 3.01  loss_cls: 1.433  loss_box_reg: 0.7118  loss_mask: 0.6747  loss_rpn_cls: 0.1206  loss_rpn_loc: 0.05998    time: 0.7884  last_time: 0.8359  data_time: 0.1610  last_data_time: 0.1876   lr: 9.9902e-06  max_mem: 3311M
[05/05 01:15:01 d2.utils.events]:  eta: 1:06:48  iter: 59  total_loss: 2.771  loss_cls: 1.22  loss_box_reg: 0.8201  loss_mask: 0.6438  loss_rpn_cls: 0.01362  loss_rpn_loc: 0.05195    time: 0.8003  last_time: 0.9899  data_time: 0.1566  last_data_time: 0.1909   lr: 1.4985e-05  max_mem: 3803M
[05/05 01:15:17 d2.utils.events]:  eta: 1:06:27  iter: 79  total_loss: 2.437  loss_cls: 0.9983  loss_box_reg: 0.7539  loss_mask: 

In [ ]:
# Save the configuration to a config.yaml file
# Save the configuration to a config.yaml file
config_dir='/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split1/kfold_split_1.coco/result-detectron2/Detectron2_Models'
if not os.path.exists(config_dir):
    os.makedirs(config_dir)
config_yaml_path = "/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split1/kfold_split_1.coco/result-detectron2/Detectron2_Models/config.yaml"
with open(config_yaml_path, 'w') as file:
    yaml.dump(cfg, file)

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_val", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "my_dataset_val")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

[05/05 02:16:44 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split1/kfold_split_1.coco/result-detectron2/OUTPUT/model_final.pth ...
[05/05 02:16:44 d2.evaluation.coco_evaluation]: Fast COCO eval is not built. Falling back to official COCO eval.
WARNING [05/05 02:16:44 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[05/05 02:16:44 d2.data.datasets.coco]: Loaded 54 images in COCO format from /content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split1/kfold_split_1.coco/valid/_annotations.coco.json
[05/05 02:16:44 d2.data.build]: Distribution of instances among all 4 categories:
|   category    | #instances   |  category  | #instances   |  category  | #instances   |
|:-------------:|:-------------|:----------:|:-------------|:----------:|:-------------|
| rice-segmen.. | 0            |   Break    | 64           |  In

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[05/05 02:16:50 d2.evaluation.evaluator]: Inference done 11/54. Dataloading: 0.2744 s/iter. Inference: 0.0947 s/iter. Eval: 0.0713 s/iter. Total: 0.4404 s/iter. ETA=0:00:18
[05/05 02:16:56 d2.evaluation.evaluator]: Inference done 25/54. Dataloading: 0.2622 s/iter. Inference: 0.0878 s/iter. Eval: 0.0439 s/iter. Total: 0.3941 s/iter. ETA=0:00:11
[05/05 02:17:01 d2.evaluation.evaluator]: Inference done 36/54. Dataloading: 0.2919 s/iter. Inference: 0.0864 s/iter. Eval: 0.0403 s/iter. Total: 0.4187 s/iter. ETA=0:00:07
[05/05 02:17:06 d2.evaluation.evaluator]: Inference done 49/54. Dataloading: 0.2866 s/iter. Inference: 0.0876 s/iter. Eval: 0.0427 s/iter. Total: 0.4171 s/iter. ETA=0:00:02
[05/05 02:17:08 d2.evaluation.evaluator]: Total inference time: 0:00:19.912143 (0.406370 s / iter per device, on 1 devices)
[05/05 02:17:08 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:04 (0.086861 s / iter per device, on 1 devices)
[05/05 02:17:08 d2.evaluation.coco_evaluation]: Prepar

### Split 2

In [ ]:
register_coco_instances("my_dataset_train2",{},"/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split2/kfold_split_2.coco/train/_annotations.coco.json","/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split2/kfold_split_2.coco/train")
register_coco_instances("my_dataset_val2",{},"/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split2/kfold_split_2.coco/valid/_annotations.coco.json","/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split2/kfold_split_2.coco/valid")
train_metadata = MetadataCatalog.get("my_dataset_train2")
train_dataset_dicts = DatasetCatalog.get("my_dataset_train2")
val_metadata = MetadataCatalog.get("my_dataset_val2")
val_dataset_dicts = DatasetCatalog.get("my_dataset_val2")

[05/05 03:28:20 d2.data.datasets.coco]: Loading /content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split2/kfold_split_2.coco/train/_annotations.coco.json takes 3.03 seconds.
WARNING [05/05 03:28:20 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[05/05 03:28:20 d2.data.datasets.coco]: Loaded 216 images in COCO format from /content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split2/kfold_split_2.coco/train/_annotations.coco.json
WARNING [05/05 03:28:21 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[05/05 03:28:21 d2.data.datasets.coco]: Loaded 54 images in COCO format from /content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split2/kfold_split_2.coco/valid/_annotations.coco.json


In [ ]:
Save_detectron2='/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split2/kfold_split_2.coco/result-detectron2'
if not os.path.exists(Save_detectron2):
    os.makedirs(Save_detectron2)
%cd /content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split2/kfold_split_2.coco/.coco/result-detectron2
output_dir='/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split2/kfold_split_2.coco/result-detectron2/OUTPUT'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
cfg = get_cfg()
cfg.OUTPUT_DIR = output_dir
cfg.merge_from_file("/content/drive/Shareddrives/Dehusked_Rice/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("my_dataset_train2",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 5000   # 1000 iterations seems good enough for this dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # Default is 512, using 256 for this dataset.
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) #Create an instance of of DefaultTrainer with the given congiguration
trainer.resume_or_load(resume=False)

[Errno 2] No such file or directory: '/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split2/kfold_split_2.coco/.coco/result-detectron2'
/content/drive/Shareddrives/Dehusked_Rice
[05/05 03:28:25 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(

model_final_f10217.pkl: 178MB [00:01, 121MB/s]                           
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


In [ ]:
trainer.train()

[05/05 03:28:31 d2.engine.train_loop]: Starting training from iteration 0


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[05/05 03:28:50 d2.utils.events]:  eta: 0:51:18  iter: 19  total_loss: 3.862  loss_cls: 1.768  loss_box_reg: 0.5331  loss_mask: 0.6897  loss_rpn_cls: 0.7834  loss_rpn_loc: 0.06964    time: 0.6052  last_time: 0.5679  data_time: 0.1123  last_data_time: 0.0119   lr: 4.9953e-06  max_mem: 2951M
[05/05 03:29:05 d2.utils.events]:  eta: 0:52:33  iter: 39  total_loss: 3.241  loss_cls: 1.6  loss_box_reg: 0.73  loss_mask: 0.6722  loss_rpn_cls: 0.1585  loss_rpn_loc: 0.06313    time: 0.6269  last_time: 0.6996  data_time: 0.0081  last_data_time: 0.0055   lr: 9.9902e-06  max_mem: 3331M
[05/05 03:29:19 d2.utils.events]:  eta: 0:54:04  iter: 59  total_loss: 2.94  loss_cls: 1.383  loss_box_reg: 0.8022  loss_mask: 0.6412  loss_rpn_cls: 0.01876  loss_rpn_loc: 0.0481    time: 0.6438  last_time: 0.6718  data_time: 0.0077  last_data_time: 0.0054   lr: 1.4985e-05  max_mem: 3587M
[05/05 03:29:33 d2.utils.events]:  eta: 0:54:36  iter: 79  total_loss: 2.621  loss_cls: 1.111  loss_box_reg: 0.816  loss_mask: 0.602

In [ ]:
# Save the configuration to a config.yaml file
# Save the configuration to a config.yaml file
config_dir='/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split2/kfold_split_2.coco/result-detectron2/Detectron2_Models'
if not os.path.exists(config_dir):
    os.makedirs(config_dir)
config_yaml_path = "/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split2/kfold_split_2.coco/result-detectron2/Detectron2_Models/config.yaml"
with open(config_yaml_path, 'w') as file:
    yaml.dump(cfg, file)

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_val2", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "my_dataset_val2")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

[05/05 04:32:09 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split2/kfold_split_2.coco/result-detectron2/OUTPUT/model_final.pth ...
[05/05 04:32:10 d2.evaluation.coco_evaluation]: Fast COCO eval is not built. Falling back to official COCO eval.
WARNING [05/05 04:32:11 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[05/05 04:32:11 d2.data.datasets.coco]: Loaded 54 images in COCO format from /content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split2/kfold_split_2.coco/valid/_annotations.coco.json
[05/05 04:32:11 d2.data.build]: Distribution of instances among all 4 categories:
|   category    | #instances   |  category  | #instances   |  category  | #instances   |
|:-------------:|:-------------|:----------:|:-------------|:----------:|:-------------|
| rice-segmen.. | 0            |   Break    | 75           |  In

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[05/05 04:32:13 d2.evaluation.evaluator]: Inference done 11/54. Dataloading: 0.0503 s/iter. Inference: 0.0872 s/iter. Eval: 0.0338 s/iter. Total: 0.1712 s/iter. ETA=0:00:07
[05/05 04:32:18 d2.evaluation.evaluator]: Inference done 36/54. Dataloading: 0.0488 s/iter. Inference: 0.0915 s/iter. Eval: 0.0545 s/iter. Total: 0.1949 s/iter. ETA=0:00:03
[05/05 04:32:22 d2.evaluation.evaluator]: Total inference time: 0:00:09.446333 (0.192782 s / iter per device, on 1 devices)
[05/05 04:32:22 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:04 (0.090827 s / iter per device, on 1 devices)
[05/05 04:32:22 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[05/05 04:32:22 d2.evaluation.coco_evaluation]: Saving results to ./output/coco_instances_results.json
[05/05 04:32:22 d2.evaluation.coco_evaluation]: Evaluating predictions with official COCO API...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
E

### Split 3

In [ ]:
register_coco_instances("my_dataset_train3",{},"/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split3/kfold_split_3.coco/train/_annotations.coco.json","/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split3/kfold_split_3.coco/train")
register_coco_instances("my_dataset_val3",{},"/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split3/kfold_split_3.coco/valid/_annotations.coco.json","/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split3/kfold_split_3.coco/valid")
train_metadata = MetadataCatalog.get("my_dataset_train3")
train_dataset_dicts = DatasetCatalog.get("my_dataset_train3")
val_metadata = MetadataCatalog.get("my_dataset_val3")
val_dataset_dicts = DatasetCatalog.get("my_dataset_val3")

[05/05 06:41:46 d2.data.datasets.coco]: Loading /content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split3/kfold_split_3.coco/train/_annotations.coco.json takes 4.13 seconds.
WARNING [05/05 06:41:46 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[05/05 06:41:46 d2.data.datasets.coco]: Loaded 216 images in COCO format from /content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split3/kfold_split_3.coco/train/_annotations.coco.json
WARNING [05/05 06:41:47 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[05/05 06:41:47 d2.data.datasets.coco]: Loaded 54 images in COCO format from /content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split3/kfold_split_3.coco/valid/_annotations.coco.json


In [ ]:
Save_detectron2='/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split3/kfold_split_3.coco/result-detectron2'
if not os.path.exists(Save_detectron2):
    os.makedirs(Save_detectron2)
%cd /content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split3/kfold_split_3.coco/.coco/result-detectron2
output_dir='/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split3/kfold_split_3.coco/result-detectron2/OUTPUT'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
cfg = get_cfg()
cfg.OUTPUT_DIR = output_dir
cfg.merge_from_file("/content/drive/Shareddrives/Dehusked_Rice/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("my_dataset_train3",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 5000   # 1000 iterations seems good enough for this dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # Default is 512, using 256 for this dataset.
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) #Create an instance of of DefaultTrainer with the given congiguration
trainer.resume_or_load(resume=False)

[Errno 2] No such file or directory: '/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split3/kfold_split_3.coco/.coco/result-detectron2'
/content/drive/Shareddrives/Dehusked_Rice
[05/05 06:47:33 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(

model_final_f10217.pkl: 178MB [00:01, 118MB/s]                           
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


In [ ]:
trainer.train()

[05/05 06:48:12 d2.engine.train_loop]: Starting training from iteration 0


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[05/05 06:48:30 d2.utils.events]:  eta: 0:45:43  iter: 19  total_loss: 3.265  loss_cls: 1.444  loss_box_reg: 0.4887  loss_mask: 0.6917  loss_rpn_cls: 0.5918  loss_rpn_loc: 0.05077    time: 0.5695  last_time: 0.5325  data_time: 0.1482  last_data_time: 0.0057   lr: 4.9953e-06  max_mem: 2828M
[05/05 06:48:48 d2.utils.events]:  eta: 0:52:51  iter: 39  total_loss: 3.14  loss_cls: 1.345  loss_box_reg: 0.7482  loss_mask: 0.6772  loss_rpn_cls: 0.2084  loss_rpn_loc: 0.07148    time: 0.6507  last_time: 0.8033  data_time: 0.0111  last_data_time: 0.0063   lr: 9.9902e-06  max_mem: 3526M
[05/05 06:49:03 d2.utils.events]:  eta: 0:55:58  iter: 59  total_loss: 2.675  loss_cls: 1.17  loss_box_reg: 0.7923  loss_mask: 0.6461  loss_rpn_cls: 0.01459  loss_rpn_loc: 0.0581    time: 0.6881  last_time: 0.5180  data_time: 0.0124  last_data_time: 0.0109   lr: 1.4985e-05  max_mem: 3656M
[05/05 06:49:18 d2.utils.events]:  eta: 0:57:14  iter: 79  total_loss: 2.436  loss_cls: 0.9578  loss_box_reg: 0.8153  loss_mask: 

In [ ]:
# Save the configuration to a config.yaml file
# Save the configuration to a config.yaml file
config_dir='/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split3/kfold_split_3.coco/result-detectron2/Detectron2_Models'
if not os.path.exists(config_dir):
    os.makedirs(config_dir)
config_yaml_path = "/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split3/kfold_split_3.coco/result-detectron2/Detectron2_Models/config.yaml"
with open(config_yaml_path, 'w') as file:
    yaml.dump(cfg, file)

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_val3", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "my_dataset_val3")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

[05/05 07:52:24 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split3/kfold_split_3.coco/result-detectron2/OUTPUT/model_final.pth ...
[05/05 07:52:25 d2.evaluation.coco_evaluation]: Fast COCO eval is not built. Falling back to official COCO eval.
WARNING [05/05 07:52:25 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[05/05 07:52:25 d2.data.datasets.coco]: Loaded 54 images in COCO format from /content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split3/kfold_split_3.coco/valid/_annotations.coco.json
[05/05 07:52:25 d2.data.build]: Distribution of instances among all 4 categories:
|   category    | #instances   |  category  | #instances   |  category  | #instances   |
|:-------------:|:-------------|:----------:|:-------------|:----------:|:-------------|
| rice-segmen.. | 0            |   Break    | 61           |  In

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[05/05 07:52:28 d2.evaluation.evaluator]: Inference done 11/54. Dataloading: 0.0963 s/iter. Inference: 0.0900 s/iter. Eval: 0.0484 s/iter. Total: 0.2348 s/iter. ETA=0:00:10
[05/05 07:52:34 d2.evaluation.evaluator]: Inference done 31/54. Dataloading: 0.1263 s/iter. Inference: 0.0889 s/iter. Eval: 0.0374 s/iter. Total: 0.2527 s/iter. ETA=0:00:05
[05/05 07:52:39 d2.evaluation.evaluator]: Inference done 51/54. Dataloading: 0.1285 s/iter. Inference: 0.0879 s/iter. Eval: 0.0357 s/iter. Total: 0.2523 s/iter. ETA=0:00:00


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[05/05 07:52:40 d2.evaluation.evaluator]: Total inference time: 0:00:12.467533 (0.254439 s / iter per device, on 1 devices)
[05/05 07:52:40 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:04 (0.088163 s / iter per device, on 1 devices)
[05/05 07:52:40 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[05/05 07:52:40 d2.evaluation.coco_evaluation]: Saving results to ./output/coco_instances_results.json
[05/05 07:52:41 d2.evaluation.coco_evaluation]: Evaluating predictions with official COCO API...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.76s).
Accumulating evaluation results...
DONE (t=0.04s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.692
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.754
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.754
 Average

### Split 4

In [ ]:
register_coco_instances("my_dataset_train4",{},"/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split4/kfold_split_4.coco/train/_annotations.coco.json","/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split4/kfold_split_4.coco/train")
register_coco_instances("my_dataset_val4",{},"/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split4/kfold_split_4.coco/valid/_annotations.coco.json","/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split4/kfold_split_4.coco/valid")
train_metadata = MetadataCatalog.get("my_dataset_train4")
train_dataset_dicts = DatasetCatalog.get("my_dataset_train4")
val_metadata = MetadataCatalog.get("my_dataset_val4")
val_dataset_dicts = DatasetCatalog.get("my_dataset_val4")

[05/05 08:04:54 d2.data.datasets.coco]: Loading /content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split4/kfold_split_4.coco/train/_annotations.coco.json takes 3.26 seconds.
WARNING [05/05 08:04:54 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[05/05 08:04:54 d2.data.datasets.coco]: Loaded 216 images in COCO format from /content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split4/kfold_split_4.coco/train/_annotations.coco.json
[05/05 08:04:55 d2.data.datasets.coco]: Loading /content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split4/kfold_split_4.coco/valid/_annotations.coco.json takes 1.20 seconds.
WARNING [05/05 08:04:55 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[05/05 08:04:55 d2.data.datasets.coco]: Loaded 54 images in COCO format from /content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/spli

In [ ]:
Save_detectron2='/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split4/kfold_split_4.coco/result-detectron2'
if not os.path.exists(Save_detectron2):
    os.makedirs(Save_detectron2)
%cd /content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split4/kfold_split_4.coco/.coco/result-detectron2
output_dir='/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split4/kfold_split_4.coco/result-detectron2/OUTPUT'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
cfg = get_cfg()
cfg.OUTPUT_DIR = output_dir
cfg.merge_from_file("/content/drive/Shareddrives/Dehusked_Rice/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("my_dataset_train4",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 5000   # 1000 iterations seems good enough for this dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # Default is 512, using 256 for this dataset.
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) #Create an instance of of DefaultTrainer with the given congiguration
trainer.resume_or_load(resume=False)

[Errno 2] No such file or directory: '/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split4/kfold_split_4.coco/.coco/result-detectron2'
/content/drive/Shareddrives/Dehusked_Rice
[05/05 08:06:19 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(

model_final_f10217.pkl: 178MB [00:01, 173MB/s]                           
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


In [ ]:
trainer.train()

[05/05 08:08:43 d2.engine.train_loop]: Starting training from iteration 0


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[05/05 08:09:00 d2.utils.events]:  eta: 0:48:49  iter: 19  total_loss: 4.104  loss_cls: 1.763  loss_box_reg: 0.5157  loss_mask: 0.691  loss_rpn_cls: 1.052  loss_rpn_loc: 0.07938    time: 0.5812  last_time: 0.6293  data_time: 0.1210  last_data_time: 0.0073   lr: 4.9953e-06  max_mem: 2913M
[05/05 08:09:16 d2.utils.events]:  eta: 0:51:43  iter: 39  total_loss: 3.178  loss_cls: 1.566  loss_box_reg: 0.7512  loss_mask: 0.6751  loss_rpn_cls: 0.06806  loss_rpn_loc: 0.05461    time: 0.6207  last_time: 0.6868  data_time: 0.0079  last_data_time: 0.0064   lr: 9.9902e-06  max_mem: 3411M
[05/05 08:09:31 d2.utils.events]:  eta: 0:52:39  iter: 59  total_loss: 2.836  loss_cls: 1.324  loss_box_reg: 0.7849  loss_mask: 0.6443  loss_rpn_cls: 0.01649  loss_rpn_loc: 0.0571    time: 0.6655  last_time: 0.7840  data_time: 0.0839  last_data_time: 0.0059   lr: 1.4985e-05  max_mem: 3533M
[05/05 08:09:45 d2.utils.events]:  eta: 0:53:20  iter: 79  total_loss: 2.495  loss_cls: 1.064  loss_box_reg: 0.7908  loss_mask: 

In [ ]:
# Save the configuration to a config.yaml file
# Save the configuration to a config.yaml file
config_dir='/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split4/kfold_split_4.coco/result-detectron2/Detectron2_Models'
if not os.path.exists(config_dir):
    os.makedirs(config_dir)
config_yaml_path = "/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split4/kfold_split_4.coco/result-detectron2/Detectron2_Models/config.yaml"
with open(config_yaml_path, 'w') as file:
    yaml.dump(cfg, file)

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_val4", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "my_dataset_val4")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

[05/05 09:13:27 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split4/kfold_split_4.coco/result-detectron2/OUTPUT/model_final.pth ...
[05/05 09:13:28 d2.evaluation.coco_evaluation]: Fast COCO eval is not built. Falling back to official COCO eval.
WARNING [05/05 09:13:28 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[05/05 09:13:28 d2.data.datasets.coco]: Loaded 54 images in COCO format from /content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split4/kfold_split_4.coco/valid/_annotations.coco.json
[05/05 09:13:28 d2.data.build]: Distribution of instances among all 4 categories:
|   category    | #instances   |  category  | #instances   |  category  | #instances   |
|:-------------:|:-------------|:----------:|:-------------|:----------:|:-------------|
| rice-segmen.. | 0            |   Break    | 57           |  In

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[05/05 09:13:30 d2.evaluation.evaluator]: Inference done 11/54. Dataloading: 0.0512 s/iter. Inference: 0.0846 s/iter. Eval: 0.0259 s/iter. Total: 0.1616 s/iter. ETA=0:00:06
[05/05 09:13:36 d2.evaluation.evaluator]: Inference done 36/54. Dataloading: 0.0641 s/iter. Inference: 0.0893 s/iter. Eval: 0.0426 s/iter. Total: 0.1961 s/iter. ETA=0:00:03


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[05/05 09:13:39 d2.evaluation.evaluator]: Total inference time: 0:00:09.479310 (0.193455 s / iter per device, on 1 devices)
[05/05 09:13:39 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:04 (0.089332 s / iter per device, on 1 devices)
[05/05 09:13:39 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[05/05 09:13:39 d2.evaluation.coco_evaluation]: Saving results to ./output/coco_instances_results.json
[05/05 09:13:39 d2.evaluation.coco_evaluation]: Evaluating predictions with official COCO API...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.86s).
Accumulating evaluation results...
DONE (t=0.06s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.647
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.704
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.704
 Average

### Split 5

In [ ]:
register_coco_instances("my_dataset_train5",{},"/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split5/kfold_split_5.coco/train/_annotations.coco.json","/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split5/kfold_split_5.coco/train")
register_coco_instances("my_dataset_val5",{},"/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split5/kfold_split_5.coco/valid/_annotations.coco.json","/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split5/kfold_split_5.coco/valid")
train_metadata = MetadataCatalog.get("my_dataset_train5")
train_dataset_dicts = DatasetCatalog.get("my_dataset_train5")
val_metadata = MetadataCatalog.get("my_dataset_val5")
val_dataset_dicts = DatasetCatalog.get("my_dataset_val5")

[05/05 09:15:35 d2.data.datasets.coco]: Loading /content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split5/kfold_split_5.coco/train/_annotations.coco.json takes 2.53 seconds.
WARNING [05/05 09:15:35 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[05/05 09:15:35 d2.data.datasets.coco]: Loaded 216 images in COCO format from /content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split5/kfold_split_5.coco/train/_annotations.coco.json
WARNING [05/05 09:15:35 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[05/05 09:15:35 d2.data.datasets.coco]: Loaded 54 images in COCO format from /content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split5/kfold_split_5.coco/valid/_annotations.coco.json


In [ ]:
Save_detectron2='/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split5/kfold_split_5.coco/result-detectron2'
if not os.path.exists(Save_detectron2):
    os.makedirs(Save_detectron2)
%cd /content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split5/kfold_split_5.coco/.coco/result-detectron2
output_dir='/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split5/kfold_split_5.coco/result-detectron2/OUTPUT'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
cfg = get_cfg()
cfg.OUTPUT_DIR = output_dir
cfg.merge_from_file("/content/drive/Shareddrives/Dehusked_Rice/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("my_dataset_train5",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 5000   # 1000 iterations seems good enough for this dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # Default is 512, using 256 for this dataset.
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) #Create an instance of of DefaultTrainer with the given congiguration
trainer.resume_or_load(resume=False)

[Errno 2] No such file or directory: '/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split5/kfold_split_5.coco/.coco/result-detectron2'
/content/drive/Shareddrives/Dehusked_Rice
[05/05 09:16:42 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(

roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


In [ ]:
trainer.train()

[05/05 09:17:05 d2.engine.train_loop]: Starting training from iteration 0


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[05/05 09:17:18 d2.utils.events]:  eta: 0:51:23  iter: 19  total_loss: 3.964  loss_cls: 1.706  loss_box_reg: 0.5735  loss_mask: 0.6949  loss_rpn_cls: 0.9438  loss_rpn_loc: 0.07622    time: 0.6194  last_time: 0.6337  data_time: 0.0405  last_data_time: 0.0056   lr: 4.9953e-06  max_mem: 3917M
[05/05 09:17:33 d2.utils.events]:  eta: 0:54:48  iter: 39  total_loss: 3.302  loss_cls: 1.589  loss_box_reg: 0.7612  loss_mask: 0.677  loss_rpn_cls: 0.1169  loss_rpn_loc: 0.06706    time: 0.6652  last_time: 0.7854  data_time: 0.0087  last_data_time: 0.0088   lr: 9.9902e-06  max_mem: 3917M
[05/05 09:17:46 d2.utils.events]:  eta: 0:54:55  iter: 59  total_loss: 2.814  loss_cls: 1.352  loss_box_reg: 0.7731  loss_mask: 0.6413  loss_rpn_cls: 0.01403  loss_rpn_loc: 0.04357    time: 0.6758  last_time: 0.7389  data_time: 0.0085  last_data_time: 0.0059   lr: 1.4985e-05  max_mem: 3917M
[05/05 09:18:01 d2.utils.events]:  eta: 0:55:31  iter: 79  total_loss: 2.602  loss_cls: 1.11  loss_box_reg: 0.8254  loss_mask: 

In [ ]:
# Save the configuration to a config.yaml file
# Save the configuration to a config.yaml file
config_dir='/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split5/kfold_split_5.coco/result-detectron2/Detectron2_Models'
if not os.path.exists(config_dir):
    os.makedirs(config_dir)
config_yaml_path = "/content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split5/kfold_split_5.coco/result-detectron2/Detectron2_Models/config.yaml"
with open(config_yaml_path, 'w') as file:
    yaml.dump(cfg, file)

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_val5", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "my_dataset_val5")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

[05/05 10:22:10 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split5/kfold_split_5.coco/result-detectron2/OUTPUT/model_final.pth ...
[05/05 10:22:11 d2.evaluation.coco_evaluation]: Fast COCO eval is not built. Falling back to official COCO eval.
WARNING [05/05 10:22:11 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[05/05 10:22:11 d2.data.datasets.coco]: Loaded 54 images in COCO format from /content/drive/Shareddrives/Dehusked_Rice/Kfold_YOLOv8_Detectron2/split5/kfold_split_5.coco/valid/_annotations.coco.json
[05/05 10:22:11 d2.data.build]: Distribution of instances among all 4 categories:
|   category    | #instances   |  category  | #instances   |  category  | #instances   |
|:-------------:|:-------------|:----------:|:-------------|:----------:|:-------------|
| rice-segmen.. | 0            |   Break    | 63           |  In

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[05/05 10:22:13 d2.evaluation.evaluator]: Inference done 11/54. Dataloading: 0.0513 s/iter. Inference: 0.0873 s/iter. Eval: 0.0404 s/iter. Total: 0.1789 s/iter. ETA=0:00:07
[05/05 10:22:19 d2.evaluation.evaluator]: Inference done 38/54. Dataloading: 0.0633 s/iter. Inference: 0.0875 s/iter. Eval: 0.0377 s/iter. Total: 0.1886 s/iter. ETA=0:00:03
[05/05 10:22:22 d2.evaluation.evaluator]: Total inference time: 0:00:09.554051 (0.194981 s / iter per device, on 1 devices)
[05/05 10:22:22 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:04 (0.087977 s / iter per device, on 1 devices)
[05/05 10:22:22 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[05/05 10:22:22 d2.evaluation.coco_evaluation]: Saving results to ./output/coco_instances_results.json
[05/05 10:22:22 d2.evaluation.coco_evaluation]: Evaluating predictions with official COCO API...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
E